# Trabalho Prático I - Problema de Transporte de Objeto

Descrição na [ementa](enunciado_trabalho_I_v2.pdf).

Neste cenário, um agente deve percorrer a grade 7x6, encontrar o objeto e transportá-lo até na base. Essa tarefa deve ser executada na menor quantidade de passos detempo possível. O agente não possui nenhum conhecimento prévio sobre o ambiente, o qual possui paredes, as quais ele não pode transpor. O agente também não possui conhecimento prévio sobre a localização do objeto. A localização inicial do agente, disposição das paredes e objeto são sempre fixas, conforme indicado na ilustração. A cada passo de tempo, o agente pode executar os seguintes movimentos na grade:
- mover para cima, baixo, esquerda ou direita;
- permanecer na mesma célula;

Este cenário apresenta algumas restrições de movimentação:
- O agente pode realizar apenas uma movimentação por passo de tempo;
- Se o agente escolher se mover para uma célula que não está vazia, seja por conta de uma parede ou objeto, ele não se move, i.e., permanece na mesma célula;
- Qualquer tentativa de locomoção para além da grade, resultará na não movimentação do agente;
- O objeto sé pode ser agarrado pela sua esquerda ou direita;
- Quando o agente ́e posicionado à direita ou esquerda do objeto, o objeto ée agarrado automaticamente;
- Uma vez agarrado o objeto, o agente não pode soltá-lo;
- O agente, quando agarrado ao objeto, só consegue se mover para uma nova célula desde que não haja nenhuma restrição de movimentação para o agente e objeto;

O episódio ée concluído automaticamente quando o objeto entra na base ou se atingir um número máximo de passos de tempo sem resolver a tarefa. Em ambos os casos, um novo episódio ́é iniciado, com o agente e objeto situados conforme abaixo.

In [5]:
# Libs necessária
import gym
from gym.envs.toy_text import discrete

import numpy as np

#import random

In [41]:
# Montando o cenário 
# Exemplo 1: https://towardsdatascience.com/creating-a-custom-openai-gym-environment-for-stock-trading-be532be3910e
# Exemplo 2: https://github.com/caburu/gym-cliffwalking/blob/master/gym_cliffwalking/envs/cliffwalking_env.py
# Exemplo 3: https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py

class ObjectTransportEnv(discrete.DiscreteEnv):
    """
    O mapa é descrito com:
    _ : caminho livre
    o : posição inicial do agente
    + : objeto a ser transportado
    $ : objetivo
    # : bloqueio
    O episódio termina chegar no objetivo.
    Sua recompensa é 1 se pegar o objeto e 1 se chegar ao objetivo com o objeto.
    Fora do mapa não é acessível. Mais regras na ementa acima.
    """

    metadata = {"render.modes": ["human", "ansi"]}
    l_free_path = b'_'
    l_agent = b'o'
    l_object = b'+'
    l_goal = b'$'
    l_wall = b'#'
    actions_movements = [
        (0, -1) # left
        ,(-1, 0) # up
        ,(0, 1) # rigth
        ,(1, 0) # down
    ]

    def _inc(self, row, col, action):
        inc_row, inc_col = action
        col = col + inc_col
        row = row + inc_row
        return (row, col)

    def _update_probability_matrix(self, predicted_pos):
        newrow, newcol = predicted_pos
        newstate = self._to_s(newrow, newcol)
        newletter = self.desc[newrow, newcol]
        done = bytes(newletter) in self.l_goal
        reward = float(newletter == self.l_goal 
            and newcol < (self.ncol-1)
            and self.desc[newrow, newcol+1] == self.l_object)
        return newstate, reward, done

    def _to_s(self, row, col):
        return row * self.ncol + col

    def _possible_actions(self, row, col):
        for index in range(len(self.actions_movements)):
            newpos = self._inc(row, col, self.actions_movements[index])
            if self._is_valid_pos(newpos):
                yield index, newpos
    
    def _is_valid_pos(self, pos):
        row, col = pos
        return row >= 0 and col >= 0 and row < self.nrow and col < self.ncol

    def __init__(self, desc):
        self.desc = np.asarray(desc, dtype="c")
        self.nrow, self.ncol = self.desc.shape
        self.reward_range = (0, 1)

        # 4 ações (cima, baixo, esquerda, direita)
        nA = len(self.actions_movements)
        # Espaço é o tamanho do mapa
        nS = self.nrow * self.ncol

        # estado inicial
        isd = np.array(self.desc == self.l_agent).astype("float64").ravel()
        isd /= isd.sum()

        # lista com transições (probability, nextstate, reward, done)
        P = {s: {a: [(0.0, s, 0.0, False)] for a in range(nA)} for s in range(nS)}
        
        for row in range(self.nrow):
            for col in range(self.ncol):
                s = self._to_s(row, col)
                # movimento protegido para não ir pra fora da lista
                #for action, predicted_pos in self._possible_actions(row, col):
                for action, predicted_pos in self._possible_actions(row, col):
                    li = P[s][action]
                    li.append((1.0, *self._update_probability_matrix(predicted_pos)))

        super(ObjectTransportEnv, self).__init__(nS, nA, P, isd)

    def render(self, mode="human"):
        outfile = StringIO() if mode == "ansi" else sys.stdout

        desc = self.desc.tolist()
        colors = {
            self.l_goal: 'blue',
            self.l_agent: 'red',
            self.l_wall: 'white',
            self.l_object: 'magenta',
        }
        for row in range(len(desc)):
            for col in range(len(desc[row])):
                sb = desc[row][col]
                color = colors.get(sb)
                ss = sb.decode('utf-8')
                desc[row][col] = gym.utils.colorize(ss, color, highlight=True) if color else ss

        if self.lastaction is not None:
            outfile.write(
                "  ({})\n".format(["Left", "Down", "Right", "Up"][self.lastaction])
            )
        else:
            outfile.write("\n")
        outfile.write("\n".join("".join(line) for line in desc) + "\n")

        if mode != "human":
            with closing(outfile):
                return outfile.getvalue()

t1_map = [
    "__$$$__",
    "___#___",
    "___+___",
    "_______",
    "##_####",
    "o_____#"
]

t1_env = ObjectTransportEnv(t1_map)
t1_env.reset()
t1_env.render()


{0: {0: [(0.0, 0, 0.0, False)], 1: [(0.0, 0, 0.0, False)], 2: [(0.0, 0, 0.0, False), (1.0, 1, 0.0, False)], 3: [(0.0, 0, 0.0, False), (1.0, 7, 0.0, False)]}, 1: {0: [(0.0, 1, 0.0, False), (1.0, 0, 0.0, False)], 1: [(0.0, 1, 0.0, False)], 2: [(0.0, 1, 0.0, False), (1.0, 2, 0.0, True)], 3: [(0.0, 1, 0.0, False), (1.0, 8, 0.0, False)]}, 2: {0: [(0.0, 2, 0.0, False), (1.0, 1, 0.0, False)], 1: [(0.0, 2, 0.0, False)], 2: [(0.0, 2, 0.0, False), (1.0, 3, 0.0, True)], 3: [(0.0, 2, 0.0, False), (1.0, 9, 0.0, False)]}, 3: {0: [(0.0, 3, 0.0, False), (1.0, 2, 0.0, True)], 1: [(0.0, 3, 0.0, False)], 2: [(0.0, 3, 0.0, False), (1.0, 4, 0.0, True)], 3: [(0.0, 3, 0.0, False), (1.0, 10, 0.0, False)]}, 4: {0: [(0.0, 4, 0.0, False), (1.0, 3, 0.0, True)], 1: [(0.0, 4, 0.0, False)], 2: [(0.0, 4, 0.0, False), (1.0, 5, 0.0, False)], 3: [(0.0, 4, 0.0, False), (1.0, 11, 0.0, False)]}, 5: {0: [(0.0, 5, 0.0, False), (1.0, 4, 0.0, True)], 1: [(0.0, 5, 0.0, False)], 2: [(0.0, 5, 0.0, False), (1.0, 6, 0.0, False)], 3

In [45]:
def Qlearning(environment, num_episodes=100, alpha=0.3, gamma=0.9, epsilon=1.0, decay_epsilon=0.1, max_epsilon=1.0, min_epsilon=0.01):
  
  # initializing the Q-table
  Q = np.zeros((environment.observation_space.n, environment.action_space.n))
  
  # additional lists to keep track of reward and epsilon values
  rewards = []
  epsilons = []

  # episodes
  for episode in range(num_episodes):
      
      # reset the environment to start a new episode
      state = environment.reset()

      # reward accumulated along episode
      accumulated_reward = 0
      
      # steps within current episode
      for step in range(100):
          # epsilon-greedy action selection
          # exploit with probability 1-epsilon
          if np.random.uniform(0, 1) > epsilon:
              action = np.argmax(Q[state,:])
              
          # explore with probability epsilon
          else:
              action = environment.action_space.sample()

          # perform the action and observe the new state and corresponding reward
          new_state, reward, done, info = environment.step(action)
          

          # update the Q-table
          Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[new_state, :]) - Q[state, action])
          
          # update the accumulated reward
          accumulated_reward += reward

          # update the current state
          state = new_state

          # end the episode when it is done
          if done == True:
              break
      
      # decay exploration rate to ensure that the agent exploits more as it becomes experienced
      epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_epsilon*episode)
      
      # update the lists of rewards and epsilons
      rewards.append(accumulated_reward)
      epsilons.append(epsilon)

  # render the environment
  environment.render()
    
  # return the list of accumulated reward along episodes
  return rewards

In [47]:
num_episodes=100
alpha=0.3
gamma=0.9
epsilon=1.0
decay_epsilon=0.1

# run Q-learning
rewards = Qlearning(t1_env, num_episodes, alpha, gamma, epsilon, decay_epsilon)

# print results
print ("Average reward (all episodes): " + str(sum(rewards)/num_episodes))
print ("Average reward (last 10 episodes): " + str(sum(rewards[-10:])/10))

  (Left)
__$$$__
___#___
___+___
_______
##_####
o_____#
Average reward (all episodes): 0.0
Average reward (last 10 episodes): 0.0
